[Tutorial](https://lightning.ai/docs/pytorch/stable/starter/introduction.html)

In [6]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L
from lightning.pytorch import Trainer, seed_everything
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset


seed_everything(42, workers=True)

Seed set to 42


42

In [7]:
def dir_to_df(PATH:os.path)-> pd.DataFrame:
    data_dict = {"image_path":[],"label":[]}
    for label in os.listdir(PATH):
        if os.path.isdir(os.path.join(PATH,label)):
            image_path = [os.path.join(os.path.join(PATH,label),i) for i in os.listdir(os.path.join(PATH,label))]
            label_list = [label]*len(image_path)
            data_dict["image_path"].extend(image_path)
            data_dict["label"].extend(label_list)

    data_df = pd.DataFrame.from_dict(data_dict)
    return data_df

In [19]:
## Make a dataframe with the label
## Train
DATASET_PATH = os.path.join("..","dataset","nature_12K","inaturalist_12K")
TRAIN_PATH = os.path.join(DATASET_PATH,'train')
VAL_PATH = os.path.join(DATASET_PATH,'val')

train_df = dir_to_df(TRAIN_PATH)
val_df = dir_to_df(VAL_PATH)
class_mapping_dict = {j:i+1 for i,j in enumerate(val_df["label"].unique())}
train_df["label_id"] = train_df["label"].map(class_mapping_dict)
val_df["label_id"] = val_df["label"].map(class_mapping_dict)

In [20]:
train_df

,image_path,label,label_id
0,..\dataset\nature_12K\inaturalist_12K\train\Am...,Amphibia,1
1,..\dataset\nature_12K\inaturalist_12K\train\Am...,Amphibia,1
2,..\dataset\nature_12K\inaturalist_12K\train\Am...,Amphibia,1
3,..\dataset\nature_12K\inaturalist_12K\train\Am...,Amphibia,1
4,..\dataset\nature_12K\inaturalist_12K\train\Am...,Amphibia,1
...,...,...,...
9995,..\dataset\nature_12K\inaturalist_12K\train\Re...,Reptilia,10
9996,..\dataset\nature_12K\inaturalist_12K\train\Re...,Reptilia,10
9997,..\dataset\nature_12K\inaturalist_12K\train\Re...,Reptilia,10
9998,..\dataset\nature_12K\inaturalist_12K\train\Re...,Reptilia,10


In [14]:
for i,j in enumerate(val_df["label"].unique()):
    print(i,j)

0 Amphibia
1 Animalia
2 Arachnida
3 Aves
4 Fungi
5 Insecta
6 Mammalia
7 Mollusca
8 Plantae
9 Reptilia


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, ):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        return image, label

In [3]:
# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

In [4]:
# define the LightningModule
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [5]:
# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

In [6]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|██████████| 9.91M/9.91M [00:07<00:00, 1.34MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 117kB/s]
100%|██████████| 1.65M/1.65M [00:02<00:00, 789kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 754kB/s]


In [7]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 50.4 K | train
1 | decoder | Sequential | 51.2 K | train
-----------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
c:\Users\yuvar\miniconda3\envs\fastapi\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 100/100 [00:01<00:00, 51.45it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 100/100 [00:02<00:00, 49.33it/s, v_num=0]
